In [3]:
import numpy as np
from time import time
from joblib import Parallel, delayed
import sys

N = 100
t = np.random.permutation(N)

print(t)

m = np.zeros((N, N))
for i in range(N):
    m[i][t[i]] = 1

print(m)


[72 73 63 45 14 35 94 64 78 82 79 56 60  2 49 59  6 20 39 29 98  9 86 84 93
 15 68 71 13  8 76  0 17 36 16 32 70 43 65  5 97 11 80 85 22 95 57 46 75 91
 40 31 52  1 92 24 88  4 26 58 62 42 48 67 10 74 81 34 96 21 87 90 69 37 66
 38  7 51 33 61 50 99 25 53 41 30 47 55 18  3 54 27 19 12 28 89 83 23 44 77]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [21]:
def h(t):
    h = 0
    for i in range(N - 1):
        c = [0,0,0,0]
        for j in range(N - i):
            # top principal diagonals
            if t[j] == j + i: c[0] += 1
            # top secondary diagonals
            if t[j] == N - j - i - 1: c[1] += 1

            if i > 0:
                # bottom principal diagonals
                if t[j + i] == j: c[2] += 1
                # bottom secondary diagonals
                if t[j + i] == N - j - 1: c[3] += 1
        h += sum([i-1 if i > 1 else 0 for i in c])

    return h


start = time()
h_t = h(t)
print("Heuristic:", h_t)
elapsed = time() - start
print("Elapsed (sec):", elapsed)


Heuristic: 51
Elapsed (sec): 0.022897720336914062


In [44]:
def sucessor(t, i, j):
    s = list(t)
    s[i], s[j] = s[j], s[i]
    return (h(s), s)


def bestSuccessor(t):
    min = sys.maxsize
    best = None
    
    successors = Parallel(n_jobs=-1)(delayed(sucessor)(t, i, j) 
                                    for i,j in zip(
                                        np.random.choice(range(N), int(N/2)), 
                                        np.random.choice(range(N), int(N/2)))
                                   )
    for s in successors:
        if s[0] < min:
            min = s[0]
            best = s
    
    return best


start = time()
print("Best successor:", bestSuccessor(t))
elapsed = time() - start
print("Elapsed (sec):", elapsed)


Best successor: (47, [72, 73, 63, 45, 14, 35, 94, 64, 78, 82, 79, 56, 60, 2, 49, 59, 6, 20, 39, 29, 98, 9, 86, 84, 40, 15, 68, 71, 13, 8, 76, 0, 17, 36, 16, 32, 70, 43, 65, 5, 97, 11, 80, 85, 22, 95, 57, 46, 75, 91, 93, 31, 52, 1, 92, 24, 88, 4, 26, 58, 62, 42, 48, 67, 10, 74, 81, 34, 96, 21, 87, 90, 69, 37, 66, 38, 7, 51, 33, 61, 50, 99, 25, 53, 41, 30, 47, 55, 18, 3, 54, 27, 19, 12, 28, 89, 83, 23, 44, 77])
Elapsed (sec): 0.9274191856384277


In [49]:
def greed(initial):
    t = list(initial)
    h_t = h(t)
    steps = 0

    while h_t > 0:
        steps += 1
        
        if steps % 10 == 0:
            print("h(t) at", steps, "steps:", h_t)

        h_t, t = bestSuccessor(t)
        
        if steps == 100*N:
            return greed(np.random.permutation(N))

    return t, steps


for i in range(100):
    x = np.random.permutation(N)
    h_x = h(x)
    if h_x < h_t:
        print(h_x)
        t = list(x)
        h_t = h_x

start = time()

solution, steps = greed(t)
print("Solution, steps:", solution, steps)

m = np.zeros((N, N))
for i in range(N):
    m[i][solution[i]] = 1
print(m)

elapsed = time() - start
print("Elapsed (sec):", elapsed)


ImportError: [joblib] Attempting to do parallel computing without protecting your import on a system that does not support forking. To use parallel-computing in a script, you must protect your main loop using "if __name__ == '__main__'". Please see the joblib documentation on Parallel for more information